# PyTorch POS Tagging

## Requirements
- PyTorch
- huggingface_hub
- datasets
- tqdm
- spacy

In [1]:
# for pip, check conda online!
%pip install datasets huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [1]:
import zipfile
import random
from functools import partial

from tqdm.auto import tqdm
import torch
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from torch.utils.data import DataLoader

print("Torch Version: ", torch.__version__)

Torch Version:  2.5.1+cu124


Loads the POS tagging dataset from the Hugging Face hub and prepares it for further processing.

In [2]:
dataset = load_dataset("batterydata/pos_tagging")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['words', 'labels'],
        num_rows: 13054
    })
    test: Dataset({
        features: ['words', 'labels'],
        num_rows: 1451
    })
})


We only have a training and a test dataset, so we use some training samples for validation:

In [3]:
# 这行代码将原始的训练集（dataset["train"]）拆分成两部分。test_size=0.1表示将10%的数据用于测试集（在这里实际上是用作验证集）
# shuffle=True表示在拆分数据集时随机打乱数据集。
# train_test_split函数将数据集分割成训练集和测试集（或在这个情况下，是验证集）
dataset_split = dataset["train"].train_test_split(test_size=0.1, shuffle=True)
# 这行将刚才拆分出的10%的数据赋值给dataset字典中的"validation"键，作为验证集
dataset["validation"] = dataset_split["test"]
# 这行将剩下的90%的数据重新赋值给dataset字典中的"train"键，作为新的训练集
dataset["train"] = dataset_split["train"]
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['words', 'labels'],
        num_rows: 11748
    })
    test: Dataset({
        features: ['words', 'labels'],
        num_rows: 1451
    })
    validation: Dataset({
        features: ['words', 'labels'],
        num_rows: 1306
    })
})


Display the first training sample:

In [4]:
print(dataset["train"][0])

{'words': ['Many', 'retailers', 'fear', 'a', 'price', 'war', 'will', 'erupt', 'if', 'cash-strapped', 'companies', 'such', 'as', 'Campeau', 'Corp.', 'slash', 'tags', 'to', 'spur', 'sales', '.'], 'labels': ['JJ', 'NNS', 'VBP', 'DT', 'NN', 'NN', 'MD', 'VB', 'IN', 'JJ', 'NNS', 'JJ', 'IN', 'NNP', 'NNP', 'VB', 'NNS', 'TO', 'VB', 'NNS', '.']}


## Data Processing

Download and unpack GloVe 300d embeddings from a zip file, build a word-to-index dictionary, and store each word's embedding vector in a list.

In [5]:
# Download and unpack the GloVe embeddings
glove = hf_hub_download("stanfordnlp/glove", "glove.6B.zip")

# There are multiple files with different dimensionality of the features in the zip archive: 50d, 100d, 200d, 300d
filename = "glove.6B.300d.txt"

word_to_index = dict()
embeddings = []
# 使用zipfile.ZipFile打开包含GloVe词嵌入的ZIP文件。
with zipfile.ZipFile(glove, "r") as f:
    for idx, line in enumerate(f.open(filename)):
        # line原来长这样：cat 0.123 -0.456 0.789 ... 0.321；split之后长这样：values = ["cat", "0.123", "-0.456", "0.789", ..., "0.321"]
        values = line.split()
        # 第一个值是单词，需要从字节串解码为UTF-8字符串
        word = values[0].decode("utf-8")
        # 将剩余的值转换为浮点数，然后创建一个PyTorch张量。
        features = torch.tensor([float(value) for value in values[1:]])
        word_to_index[word] = idx
        embeddings.append(features)

Add padding and unknown tokens:

In [6]:
# Last token in the vocabulary is '<unk>' which is used for out-of-vocabulary words
# We also add a '<pad>' token to the vocabulary for padding sequences
'''这里添加了一个特殊的<pad>标记到word_to_index字典中，并将其ID设置为字典的当前长度。
这个标记通常用于填充序列，使它们具有相同的长度。'''
word_to_index["<pad>"] = len(word_to_index)
padding_token_id = word_to_index["<pad>"]
unk_token_id = word_to_index["<unk>"]
# 为<pad>标记添加一个全零向量作为其嵌入。这个向量的维度与其他词的嵌入相同。
embeddings.append(torch.zeros(embeddings[0].shape))
'''背景：
        1.embeddings是一个列表，其中每个元素都是一个表示单个词的嵌入向量的张量。
        2.每个嵌入向量都是一个1D张量，比如shape可能是(300,)，表示300维的词嵌入。
        3.列表中的元素数量等于词汇表的大小（包括特殊标记如<pad>和<unk>）。
   torch.stack() 函数的作用：
        1.假设我们有10000个词，每个词的嵌入是300维的。
        2.在堆叠之前，我们有一个包含10000个形状为(300,)的张量的列表。
        3.堆叠后，我们得到一个形状为(10000, 300)的2D张量。 '''
# Convert the list of tensors to a single tensor
embeddings = torch.stack(embeddings)
# embeddings.size(1) 返回的是 embeddings 张量的第二个维度的大小
# size()是 PyTorch 张量的一个方法，用于获取张量的维度信息;当不带参数调用时（如 embeddings.size()），它返回一个包含所有维度大小的元组
print(f"Embedding shape: {embeddings.size(1)}")
print(f"Padding token id: {padding_token_id}")
print(f"Unknown token id: {unk_token_id}")

Embedding shape: 300
Padding token id: 400001
Unknown token id: 400000


Create dictionaries to map labels to indices and vice versa, and print the number of unique classes.

In [7]:
'''双重循环的列表推导式解读：
result = []
for sample in dataset["train"]:
    for label in sample["labels"]:
        result.append(label)'''
# 这行代码使用列表推导和集合来获取训练集中所有唯一的标签。
# 它遍历训练集中的每个样本，收集所有标签，然后使用set()去重，最后转换回列表
labels_unique = list(
    set([label for sample in dataset["train"] for label in sample["labels"]])
)
print(labels_unique)
print(f"Number of classes: {len(labels_unique)}")
'''enumerate(labels_unique) 会产生如下的序列:
(0, 'O')(1, 'B-PER')(2, 'I-PER')(3, 'B-ORG')'''
ctoi = {label: idx for idx, label in enumerate(labels_unique)}
itoc = {idx: label for label, idx in ctoi.items()}
print(ctoi)
print(itoc)

[')', "''", 'NNP', 'WP', 'TO', 'POS', 'NNPS', '(', 'EX', 'PDT', 'VB', 'FW', 'RBR', 'RP', 'MD', 'JJS', 'VBG', '-LRB-', 'NN', 'VBZ', '#', 'WDT', '``', '.', 'RBS', 'JJ', 'VBN', 'VBD', 'LS', '-NONE-', 'IN', 'DT', ':', '-RRB-', 'NNS', 'WRB', 'JJR', 'UH', 'RB', 'PRP$', '$', 'SYM', 'WP$', 'CC', 'CD', ',', 'PRP', 'VBP']
Number of classes: 48
{')': 0, "''": 1, 'NNP': 2, 'WP': 3, 'TO': 4, 'POS': 5, 'NNPS': 6, '(': 7, 'EX': 8, 'PDT': 9, 'VB': 10, 'FW': 11, 'RBR': 12, 'RP': 13, 'MD': 14, 'JJS': 15, 'VBG': 16, '-LRB-': 17, 'NN': 18, 'VBZ': 19, '#': 20, 'WDT': 21, '``': 22, '.': 23, 'RBS': 24, 'JJ': 25, 'VBN': 26, 'VBD': 27, 'LS': 28, '-NONE-': 29, 'IN': 30, 'DT': 31, ':': 32, '-RRB-': 33, 'NNS': 34, 'WRB': 35, 'JJR': 36, 'UH': 37, 'RB': 38, 'PRP$': 39, '$': 40, 'SYM': 41, 'WP$': 42, 'CC': 43, 'CD': 44, ',': 45, 'PRP': 46, 'VBP': 47}
{0: ')', 1: "''", 2: 'NNP', 3: 'WP', 4: 'TO', 5: 'POS', 6: 'NNPS', 7: '(', 8: 'EX', 9: 'PDT', 10: 'VB', 11: 'FW', 12: 'RBR', 13: 'RP', 14: 'MD', 15: 'JJS', 16: 'VBG', 1

Map tokens and labels to indices, and prepare the dataset for training.

In [8]:
# mapping: 一个字典，用于查找映射关系;default: 可选参数，当键不在字典中时返回的默认值
# 这个函数遍历 keys 列表中的每个元素，并尝试在 mapping 字典中查找对应的值
# 返回值： 返回一个新的列表，包含所有映射后的值
def map_list_using_dict(mapping, keys: list, default=None):
    return [mapping.get(key, default) for key in keys]


# 返回值： 返回一个整数列表，表示输入单词列表中每个单词对应的索引
def map_tokens_to_indices(tokens: list[str]):
    # Return the index of each token or the index of the '<unk>' token if a token is not in the vocabulary
    return map_list_using_dict(
        word_to_index, [token.lower() for token in tokens], unk_token_id
    )


# 返回一个整数列表，表示输入标签列表中每个标签对应的索引。
def map_labels_to_indices(labels: list):
    # TODO: Implement the mapping of the labels to indices
    return map_list_using_dict(
        ctoi, [label for label in labels]
    )


# 函数功能： 对数据集中的每个样本进行处理，将单词转换为token索引，将标签转换为标签索引。
def prepare_dataset(dataset):
    # return map(lambda x: {"token_ids": map_text_to_indices(x["words"])}, dataset)
    # dataset.map() 方法是 Hugging Face 的 Dataset 类中的一个方法；它的作用是对数据集中的每个元素应用一个函数，从而转换整个数据集
    # lambda是python中的匿名函数，x是这个函数的参数，代表数据集中的一个样本
    # lambda x 函数作为参数传递给 dataset.map()。
    # dataset.map() 会遍历数据集中的每个样本，并对每个样本调用这个 lambda 函数。
    # x 代表 dataset 中的一个样本
    dataset = dataset.map(
        lambda x: {
            # x["words"] 被传递给 map_tokens_to_indices 函数，将单词列表转换为token索引列表
            "token_ids": map_tokens_to_indices(x["words"]),
            # x["labels"] 被传递给 map_labels_to_indices 函数，将标签列表转换为标签索引列表
            "label_ids": map_labels_to_indices(x["labels"]),
        },
        # num_proc=1: 指定使用1个进程进行处理。这可能是为了确保处理的顺序性或避免多进程带来的潜在问题。
        num_proc=1,
    )
    return dataset
# 返回处理后的数据集，其中每个样本都包含 "token_ids" 和 "label_ids" 字段

dataset = prepare_dataset(dataset)
dataset_train_tokenized = dataset["train"]
dataset_validation_tokenized = dataset["validation"]

# Print the first sample in the tokenized training dataset
print(dataset_train_tokenized[0].keys())
print(dataset_train_tokenized[0])

Map:   0%|          | 0/11748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1451 [00:00<?, ? examples/s]

Map:   0%|          | 0/1306 [00:00<?, ? examples/s]

dict_keys(['words', 'labels', 'token_ids', 'label_ids'])
{'words': ['Many', 'retailers', 'fear', 'a', 'price', 'war', 'will', 'erupt', 'if', 'cash-strapped', 'companies', 'such', 'as', 'Campeau', 'Corp.', 'slash', 'tags', 'to', 'spur', 'sales', '.'], 'labels': ['JJ', 'NNS', 'VBP', 'DT', 'NN', 'NN', 'MD', 'VB', 'IN', 'JJ', 'NNS', 'JJ', 'IN', 'NNP', 'NNP', 'VB', 'NNS', 'TO', 'VB', 'NNS', '.'], 'token_ids': [109, 5192, 1655, 7, 626, 136, 43, 20454, 83, 168152, 337, 125, 19, 149818, 1018, 9421, 15648, 4, 8217, 526, 2], 'label_ids': [25, 34, 47, 31, 18, 18, 14, 10, 30, 25, 34, 25, 30, 2, 2, 10, 34, 4, 10, 34, 23]}


We again pad inputs to the maximum sequence length in the batch.\
But this time, we also have to pad the labels:

In [9]:
# batch: 输入的批次，通常是一个列表，包含多个样本（每个样本是一个字典）
def pad_inputs(batch, keys_to_pad=["token_ids", "label_ids"], padding_value=-1):
    # Pad keys_to_pad to the maximum length in batch
    
    #创建一个新的字典来存储填充后的批次
    padded_batch = {}
    for key in keys_to_pad:
        # Get maximum length in batch
        # key 是当前正在处理的特征名称，例如 "token_ids" 或 "label_ids"。
        max_len = max([len(sample[key]) for sample in batch])
        # Pad all samples to the maximum length
        padded_batch[key] = torch.tensor(
            [
                sample[key] + [padding_value] * (max_len - len(sample[key]))
                for sample in batch
            ]
        )
    # Add remaining keys to the batch
    # 对于不需要填充的键：直接将所有样本的该键值转换为张量，不进行填充。
    for key in batch[0].keys():
        if key not in keys_to_pad:
            padded_batch[key] = torch.tensor([sample[key] for sample in batch])
    return padded_batch


def get_dataloader(dataset, batch_size=32, shuffle=False):
    # Create a DataLoader for the dataset
    return DataLoader(
        dataset,
        batch_size=batch_size,
        # 用于处理和组合样本列表以形成小批量的函数
        # 使用 partial 函数创建一个新的函数，这个新函数基于 pad_inputs，但预设了 padding_value 参数为 padding_token_id。
        # 这意味着当 DataLoader 调用这个函数来创建批次时，它会使用 pad_inputs 来处理可变长度的序列，并用 padding_token_id 进行填充。
        # partial允许我们基于一个已有的函数创建一个新的函数，同时预设一些参数；这在很多情况下都非常有用，特别是当你需要多次使用同一个函数，但每次只改变其中的一部分参数时。
        # 为什么需要把padding_value设为padding_token_id
        # padding_token_id 通常是词汇表中专门为填充保留的一个特殊标记的ID
        # 这里的padding_token_id是词汇表中的最后一个ID
        collate_fn=partial(pad_inputs, padding_value=padding_token_id),
        shuffle=shuffle,
    )


# Create a DataLoader for the training dataset with the selected columns
dataloader_train = get_dataloader(
    # 这行代码使用with_format方法选择了dataset_train_tokenized数据集中的"token_ids"和"label_ids"两列。
    # 这样可以确保在后续处理中只使用这两列数据。
    dataset_train_tokenized.with_format(columns=["token_ids", "label_ids"]),
    batch_size=8,
    shuffle=True,
)
dataloader_validation = get_dataloader(
    dataset_validation_tokenized.with_format(columns=["token_ids", "label_ids"]),
    batch_size=8,
    shuffle=True,
)

for batch in dataloader_train:
    token_ids = batch["token_ids"]
    labels = batch["label_ids"]
    print(token_ids)
    print(labels)
    break

tensor([[     0,    316,  35091,     16,     20,     43,    255,   2423,    689,
            548,      5,   1748,    604,    182,    147,      2, 400001, 400001,
         400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001,
         400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001],
        [  4212,   2952,      1,     42,    919,    657,     93,   1405,    544,
             22,    800, 400000,    183,    131,     29,  19979,    129,    321,
             21,   5330,   7764,      1,    717,    135, 400000,      4,    800,
              2, 400001, 400001, 400001, 400001, 400001, 400001, 400001, 400001],
        [  1995,  70914,      9,  83674,    807,  35718,    573,     55,   1424,
              3, 400000,   1356,    105,     42,   1995,  70914,      1,    114,
           5599,     82,    167,      1,    675,    322,    509,     75,      0,
             96,   8163,   1977,    513,      6,      0,   1332,   4079,      2],
        [ 28740,      9, 

## Using GPUs

So far, we have not paid attention to which device the PyTorch operations are running on.\
By default, they run on the CPU, however, a GPU is usually much faster when performing tensor operations.\
For this, you will need to have a supported GPU available on the device where you execute this code.\
Our servers at the IMS provide GPUs (strauss, nandu, kiwi).\
You can either remotely connect your editor and run the code there, or connect to a remote Python Kernel.

Once there is a supported GPU available on your machine that runs the code, you can copy tensors and even models using the method `.to(device)` to `device`.\
`device` can be specified using `torch.device`:
```python
# 'cuda' for GPU (optionally specify device id, e.g., 'cuda:0' for the first GPU) and 'cpu' for CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
```

> Note that not specifying a device id might use all GPUs! Therefore, always set the device id or restrict the available GPUs, e.g., using the environment variable `CUDA_VISIBLE_DEVICES`.
> You can set this variable first using `export CUDA_VISIBLE_DEVICES=3` so that any executed command afterward will use the GPU with id 3 (the 4th GPU) or directly set it for your command using `CUDA_VISIBLE_DEVICES=3 command`.

You may also allocate tensors on a specific device during intialization:
```python
a = torch.tensor(..., device=device)
```
This works for all the tensor creation operations!

In [10]:
print(f"CUDA available: {torch.cuda.is_available()}")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA available: True


## Our neural network consists of one fully connected linear layer

The softmax is part of the loss function in PyTorch, so you can omit this in the forward function.

The embedding layer
- maps from indices to vectors
- is not trained (freezed)

In [11]:
class SimplePOSTagger(torch.nn.Module):
    # this resembles a really simple neural network: an embedding layer followed by a fully
    # connected linear layer such that predictions are computed for each token in the sequence
    # and batch independently
    # embedding_vectors: 预训练的词嵌入向量;num_classes: 输出类别的数量（即可能的词性标签数量）;hidden_dim: 隐藏层的维度
    def __init__(self, embedding_vectors, num_classes, hidden_dim):
        super().__init__()
        # PyTorch's embedding layer maps from indices to embeddings, freeze will tell PyTorch to
        # not train this layer, i.e. not modifying any weight
        # 这行代码创建了一个预训练的嵌入层
        # torch.nn.Embedding.from_pretrained(): 这是PyTorch中创建预训练嵌入层的方法。
        # 它允许你使用已经训练好的词嵌入来初始化Embedding层。
        # freeze=True: 这个参数设置为True表示在训练过程中，这些嵌入权重将不会被更新。这通常用于保持预训练嵌入的原始信息，特别是当你认为这些预训练嵌入已经足够好，或者你的训练数据较少时
        self.embedding = torch.nn.Embedding.from_pretrained(
            embedding_vectors, freeze=True
        )
        # a fully connected linear layer mapping the embedded vector to a vector of fixed size
        # (num_classes in this case)
        # 一个全连接层，将嵌入向量映射到指定的隐藏维度，然后再映射到输出类别的数量（即可能的词性标签数量）
        # embedding_vectors.size(1) 是嵌入向量的维度，即嵌入向量中每个词的维度。
        self.hidden_layer = torch.nn.Linear(embedding_vectors.size(1), hidden_dim)
        # 另一个全连接层，将隐藏层的输出映射到类别数量的维度
        self.output_layer = torch.nn.Linear(hidden_dim, num_classes)

    def forward(self, inputs):
        # simple forwarding through our model
        # PyTorch takes care of keeping track of the operations for the backward pass
        emmedded_inputs = self.embedding(inputs)
        z_1 = self.hidden_layer(emmedded_inputs)
        # 使用 Leaky ReLU 激活函数，负斜率为 0.2 d
        '''Leaky ReLU：
定义：f(x) = x if x > 0 else αx; 其中 α 是一个小的正数，通常在 0.01 到 0.2 之间; 在代码中，α 被称为 negative_slope（负斜率）
Leaky ReLU 的工作原理：对于正输入：和 ReLU 一样，直接输出该值; 对于负输入：输出一个很小的负值（输入乘以 α）'''
        a_1 = torch.nn.functional.leaky_relu(z_1, negative_slope=0.2)
        z_2 = self.output_layer(a_1)
        return z_2  # softmax is applied in the loss function

## Set up model, loss and optimizer
- Cross Entropy is Softmax + Negative Log Likelihood
- As optimizer we use Adam (adapts the learning rate per weight)

(run this only once as Jupyter keeps the model (including the weights) and the optimizer in memory)

In [12]:
# Set up model and optimizer and move model to device
model = SimplePOSTagger(embedding_vectors=embeddings, num_classes=len(ctoi), hidden_dim=128).to(DEVICE)
# ignore_index=padding_token_id: 在计算损失时忽略填充标记，这对于处理变长序列很重要
# reduction 参数决定了如何汇总单个样本的损失值。在我们的例子中，我们使用'mean'，这意味着将所有样本的损失值相加，然后除以样本数。
criterion = torch.nn.CrossEntropyLoss(reduction='mean', ignore_index=padding_token_id)
# 使用 Adam 优化器；model.parameters(): 优化模型的所有可训练参数；lr=0.01: 设置学习率为 0.01
# Adam（Adaptive Moment Estimation）优化器是一种广泛使用的深度学习优化算法。它结合了其他几种优化算法的优点，特别适合处理大规模数据和参数。
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# 创建一个字典来存储训练过程中的损失和准确率；初始值设为 '------'，可能用于后续的打印或记录
metric_dict = {'loss': '------', 'accuracy': '------'}

## Evaluation function comparing prediction with gold label

In [13]:
def evaluate(data_iter, model):
    # data_iter：数据集的迭代器；model：要评估的模型
    correct_count = 0 # 正确预测的数量
    total_count = 0 # 总的预测数量
    for i, batch in enumerate(data_iter):
        # extract input and labels

        # move data to device since our model is on the device
        token_ids = batch["token_ids"].to(device=DEVICE)
        labels = batch["label_ids"].to(device=DEVICE)

        # predict only
        # 在预测阶段，我们不需要计算梯度，因此使用torch.no_grad()上下文管理器可以节省内存和计算资源。
        with torch.no_grad():
            outputs = model(token_ids)
        '''outputs 的结构:
            这是模型的原始输出。
            通常形状为 (batch_size, sequence_length, num_classes)。
            每个位置包含一个概率分布，表示该位置属于每个类别的可能性'''
        # torch.argmax() 或 tensor.argmax() 返回指定维度上最大值的索引。
        # dim=2 表示在第三个维度（类别维度）上进行操作
        outputs_classes = outputs.argmax(dim=2)

        # compute amount of correct predictions
        # sequence lengths within the batch might be different, so we need to take care of that
        # 这行代码的目的是计算每个序列的实际长度，忽略填充标记。
        # token_ids 每行代表一个序列，可能包含实际标记和填充标记。
        # 400001 是填充标记的 ID
        # 这是一个布尔运算，对 token_ids 中的每个元素进行。结果是一个布尔张量，与 token_ids 形状相同。true是1，false是0
        # .sum(dim=1)在第二个维度（序列长度维度）上求和；这会将每个序列的布尔值相加，得到每个序列中非填充标记的数量
        # inputs_lengths结果是一个一维张量，长度等于批次大小。每个元素表示对应序列的实际长度（不包括填充）
        inputs_lengths = (token_ids != 400001).sum(dim=1)
        
        # 这行代码的目的是计算所有序列的实际长度之和。
        total_count += inputs_lengths.sum()
        # iterate over each sample of the batch
        batch_size = outputs_classes.size(0)
        # 外层循环遍历批次中的每个序列
        for i in range(batch_size):
            # 内层循环遍历当前序列中的每个有效标记
            # inputs_lengths[i] 是第 i 个序列的实际长度（不包括填充）
            for j in range(inputs_lengths[i]):
                # outputs_classes[i][j] 是模型对第 i 个序列中第 j 个标记的预测
                # labels[i][j] 是第 i 个序列中第 j 个标记的真实标签
                # int(...) 将布尔值转换为整数（True 变为 1，False 变为 0）
                correct_count += int(outputs_classes[i][j] == labels[i][j])
    # total_count是一个 PyTorch 张量，存储了总的标记数量；.float()将张量转换为浮点类型
    # 这个方法将单元素张量转换为标准的 Python 数值类型，以便进行除法运算。
    return correct_count / total_count.float().item()

In [14]:
accuracy = evaluate(dataloader_validation, model)
print(f"Accuracy on the validation dataset: {accuracy}")

Accuracy on the validation dataset: 0.00797896731986182


## The actual training loop

- runs several epochs
- in each epoch
 - forward the batch
 - computes the loss for the output of the whole batch
 - reduces (e.g. average, sum) the loss
 - computes derivatives of weights by backpropagation
 - optimizer updates weights
 - evaluate on validation/development dataset

In [24]:
NUM_EPOCHS = 5

# a nice progress bar to make the waiting time much better
pbar = tqdm(total=NUM_EPOCHS*len(dataloader_train), postfix=metric_dict)

# evaluate on validation set first
# 这行代码是用来更新和格式化验证集上的准确率，并将其添加到 metric_dict 字典中
# evaluate(dataloader_validation, model)函数调用会返回一个 0 到 1 之间的浮点数，表示模型在验证集上的准确率
# 100 * ...将准确率转换为百分比形式。
# :6.2f：6 表示总字段宽度为 6 个字符；.2 表示保留 2 位小数。最终结果可能看起来像 " 75.00%
# {'accuracy': ...}创建一个字典，键为 'accuracy'，值为格式化后的准确率字符串
# metric_dict.update(...)将新创建的字典合并到 metric_dict 中。如果 metric_dict 已经有 'accuracy' 键，它会被更新；如果没有，会被添加。
metric_dict.update({'accuracy': f'{100*evaluate(dataloader_validation, model):6.2f}%'})
# pbar是一个 tqdm 对象，代表一个进度条。它在之前的代码中被创建，用于可视化训练进度。用于在进度条后面设置额外的信息（postfix）。这些信息会在进度条的右侧显示
# metric_dict字典包含了我们想要显示的指标，通常包括像 'loss'（损失）和 'accuracy'（准确率）这样的键值对
pbar.set_postfix(metric_dict)

# run for NUM_EPOCHS epochs
for epoch in range(NUM_EPOCHS):
    # run for every data (in batches) of our iterator
    # 设置进度条描述为当前轮数和总轮数
    pbar.set_description(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
    # 循环遍历训练数据的每个批次
    for i, batch in enumerate(dataloader_train):
        # extract input and labels and move data to device since our model is on the device
        token_ids = batch["token_ids"].to(device=DEVICE)
        labels = batch["label_ids"].to(device=DEVICE)

        # forward + backward + optimize
        #前向传播
        outputs = model(token_ids)
        
        # 2D loss function expects input as (batch, prediction, sequence) and target as (batch, sequence) containing the class index
        # criterion是预先定义的损失函数，通常是交叉熵损失（CrossEntropyLoss）用于分类任务
        # outputs是模型的原始输出；对于序列标注任务，其形状通常是 (batch_size, sequence_length, num_classes)；每个位置包含一个概率分布，表示该位置属于每个类别的可能性
        # permute 方法用于重新排列张量的维度；这样操作后，输出的形状变为 (batch_size, num_classes, sequence_length)
        '''为什么需要 permute：
            1.PyTorch 的 CrossEntropyLoss 期望输入的形状是 (batch_size, num_classes, *)，其中 * 表示任意数量的其他维度。
            2.在序列标注任务中，我们需要将 num_classes 维度移到第二位，以符合损失函数的要求。'''
        loss = criterion(outputs.permute(0,2,1), labels)
        # otherwise use view function to get rid of sequence dimension by effectively concatenating all sequence items
        # loss = criterion(outputs.view(-1, len(classes)), labels.view(-1))

        # zero the parameter gradients
        # 清除之前的梯度
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新模型参数
        optimizer.step()

        # print statistics
        pbar.update()
        # .item() 方法从张量中提取数值，将其转换为 Python 标量（如 float）。这是必要的，因为我们需要一个普通的 Python 数值来格式化字符串
        metric_dict.update({'loss': f'{loss.item():6.3f}'})
        pbar.set_postfix(metric_dict)
        
    # evaluate on validation set after each epoch
    metric_dict.update({'accuracy': f'{100*evaluate(dataloader_validation, model):6.2f}%'})
    pbar.set_postfix(metric_dict)

  0%|          | 0/7345 [00:00<?, ?it/s, accuracy=------, loss=------]

## Randomly predict sample from test set

In [ ]:
def map_indices_to_labels(indices: list):
    return map_list_using_dict(itoc, indices)

In [ ]:
# Randomly select a sample from the validation dataset
# random.choice用于从序列（如列表、元组或字符串）中随机选择一个元素
sample = random.choice(dataset_validation_tokenized)
print(sample)
# build input vector and add batch dimension
# unsqueeze() 的基本功能：是 PyTorch 中的一个张量操作，用于在指定位置添加一个新的维度。新维度的大小为 1。
# dim=0 参数：指定在张量的第 0 维（最外层）添加新维度
'''sample["token_ids"])是一个列表
torch.tensor(sample["token_ids"]) 创建一个形状为 [sequence_length] 的一维张量
1.列表：token_ids = [5, 8, 2, 10, 3, 1]
2.一维张量：token_ids = tensor([5, 8, 2, 10, 3, 1])
3.二维张量：token_ids = tensor([[5, 8, 2, 10, 3, 1]])'''
sample_tensor = torch.tensor(sample["token_ids"]).unsqueeze(dim=0).to(DEVICE)
'''unsqueeze(dim=0) 将形状从 [sequence_length] 变为 [1, sequence_length]，模拟一个批次大小为 1 的输入。
这一步是为了满足模型输入的要求。大多数深度学习模型期望输入是批次（batch）形式，即使是单个样本。
处理完成后，我们不再需要批次维度，特别是在处理单个样本时;
squeeze(dim=0) 移除第一个维度（批次维度），将形状从 [1, sequence_length, num_classes] 变为 [sequence_length, num_classes]'''
# forward / predict
with torch.no_grad():
    # get rid of batch dimension (is set to 1)
    outputs = model(sample_tensor).squeeze(dim=0)
# 每个 output 是一个表示单个位置所有类别概率的向量
# argmax() 函数找出张量中最大值的索引。dim=0 表示在第一个维度（即类别维度）上寻找最大值
# .item() 将单元素张量转换为 Python 标量
'''对 outputs 中的每个 output 执行上述操作。
结果是一个列表，包含每个位置的预测标签。'''
predictions = [itoc[output.argmax(dim=0).item()] for output in outputs]
# print() 函数可以接受多个参数。这些参数之间用逗号分隔。
print("Input:", ' '.join(sample["words"]))
print(f"Prediction:   {predictions}")
print(f"Ground truth: {sample['labels']}")
# zip() 函数将 predictions 和 sample["labels"] 这两个列表对应位置的元素配对
# 遍历 zip() 生成的每对元素；pred 是预测标签，gt 是真实标签；if pred == gt 检查预测是否正确；如果预测正确（pred == gt），则生成一个 1；否则不生成任何值
accuracy = sum([1 for pred, gt in zip(predictions, sample["labels"]) if pred == gt]) / len(sample["labels"])
print(f"Accuracy: {accuracy*100:.2f}%")

{'words': ['The', 'building', 'houses', 'about', '4,500', 'Chase', 'workers', ',', 'most', 'of', 'whom', 'will', 'be', 'moved', 'to', 'downtown', 'Brooklyn', 'after', 'the', 'bank', "'s", 'new', 'back', 'office', 'center', 'is', 'completed', 'in', '1993', '.'], 'labels': ['DT', 'NN', 'NNS', 'IN', 'CD', 'NNP', 'NNS', ',', 'RBS', 'IN', 'WP', 'MD', 'VB', 'VBN', 'TO', 'NN', 'NNP', 'IN', 'DT', 'NN', 'POS', 'JJ', 'JJ', 'NN', 'NN', 'VBZ', 'VBN', 'IN', 'CD', '.'], 'token_ids': [0, 447, 1631, 59, 14298, 4212, 537, 1, 96, 3, 1231, 43, 30, 554, 4, 2522, 4430, 49, 0, 231, 9, 50, 137, 283, 313, 14, 1315, 6, 1251, 2], 'label_ids': [31, 18, 34, 30, 44, 2, 34, 45, 24, 30, 3, 14, 10, 26, 4, 18, 2, 30, 31, 18, 5, 25, 25, 18, 18, 19, 26, 30, 44, 23]}
Input: The building houses about 4,500 Chase workers , most of whom will be moved to downtown Brooklyn after the bank 's new back office center is completed in 1993 .
Prediction:   ['DT', 'NN', 'NNS', 'IN', 'CD', 'NNP', 'NNS', ',', 'RBS', 'IN', 'WP', 'MD', '

## Interactive prediction

Note that we did not have to tokenize our data so far since tokens were given.\
For tokenizing text, you can again use the tokenization from the sentiment analysis task, but it has some trouble:

In [18]:
def tokenize_simple(text: str):
    return text.lower().split()

print(tokenize_simple("This is a simple text."))

['this', 'is', 'a', 'simple', 'text.']


Punctuation is not properly split, but for POS tagging to work correctly, we need punctuation is separate tokens too.\
We can extract words and punctuation using a regular expression (regex):

In [19]:
import re

def tokenize_regex(text: str):
    return re.findall(r"[\w']+|[.,!?;]", text.lower())

print(tokenize_regex("This is a simple text."))

['this', 'is', 'a', 'simple', 'text', '.']


There are also packages like spacy that help you with tokenization.\
We have to install it first and then download some files for the tokenizer:

In [20]:
# install spacy using pip
%pip install spacy

   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------- ----------------------- 5.0/12.2 MB 25.2 MB/s eta 0:00:01
   ------------------------------- -------- 9.7/12.2 MB 24.2 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.2 MB 22.9 MB/s eta 0:00:01
   ---------------------------------------- 12.2/12.2 MB 19.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 22.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   --------------------------------------- 632.6/632.6 kB 11.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 15.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ----------------------------------- ---- 5.5/6.2 MB 27.9 MB/s eta 0:00:01
   ------------------------

In [1]:
# download resources for english
# `run` has to be replaced by `python` if run in a shell
%run -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

def tokenize(text: str):
    # token.text 是 spaCy 库中 Token 对象的一个属性，用于获取标记（token）的原始文本形式
    return [token.text for token in nlp(text.lower())]

print(tokenize("This is a simple text."))

['this', 'is', 'a', 'simple', 'text', '.']


We render a nice text box:

In [3]:
from ipywidgets import widgets
from IPython.display import display

sentence_widget = widgets.Text(
    value="This movie is terrible",
    placeholder="Type something",
    description="Sentence:",
    disabled=False,
)
display(sentence_widget)

Text(value='This movie is terrible', description='Sentence:', placeholder='Type something')

: 

### Task
Prepare the input, and feed it through the model.

In [ ]:
text = sentence_widget.value

# convert text to token ids

# build input vector and add batch dimension


# forward / predict
with torch.no_grad():
    

# print prediction